### A  demonstration of sims_movingObjects ###

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.movingObjects as mo
import lsst.sims.maf.db as db
from lsst.sims.maf.batches import getColMap

For most typical use of sims_movingObjects, you just need to use the script 'makeLSSTobs.py' :

```
(lsst-scipipe) [ewok:~] lynnej% makeLSSTobs.py --help

usage: makeLSSTobs.py [-h] [--orbitFile ORBITFILE] [--outDir OUTDIR]
                      [--obsFile OBSFILE] [--sqlConstraint SQLCONSTRAINT]
                      [--noCamera] [--rFov RFOV]
                      [--interpolation INTERPOLATION] [--obscode OBSCODE]
                      [--tStep TSTEP] [--ephMode EPHMODE]
                      opsimDb

Generate moving object detections.

positional arguments:
  opsimDb               Opsim run db file

optional arguments:
  -h, --help            show this help message and exit
  --orbitFile ORBITFILE
                        File containing the moving object orbits.
  --outDir OUTDIR       Output directory for moving object detections.
  --obsFile OBSFILE     Output file name for moving object observations.
                        Default will build outDir/opsimRun_orbitFile_obs.txt.
  --sqlConstraint SQLCONSTRAINT
                        SQL constraint to use to select data from opsimDb.
                        Default no constraint.
  --noCamera            Do not use the LSST camera footprint, use rFov
                        instead.
  --rFov RFOV           If not using camera footprint, use a circular FOV with
                        radius rFOV degrees. Default 1.75 degrees.
  --interpolation INTERPOLATION
                        Type of interpolation between ephemerides to use.
                        Options include linear, chebyshev, and direct (no
                        interpolation). Default is linear.
  --obscode OBSCODE     Obscode for generating observations with linear or
                        direct interpolation. Default is I11 (Cerro Pachon).
  --tStep TSTEP         Timestep between ephemeris generation / linear
                        interpolation steps (in days). Relevant for linear
                        interpolation only! Default 2 hours.
  --ephMode EPHMODE     2body or nbody mode for ephemeris generation. Default
                        is nbody.
```

This runs the steps below, as a whole, to produce an output file with the observations.


But let's break down a little bit of what's happening behind the scenes.

### Reading the orbit file.

In [5]:
# First the orbit file - this is read into an Orbits object.
orbitDir = '.'
orbitfile = 'neos_100.s3m'

orbits = mo.Orbits()
orbits.readOrbits(os.path.join(orbitDir, orbitfile))

In [4]:
# The actual orbits are checked to see if they contain required columns: 
for orbit_type in orbits.dataCols:
    print(orbit_type, orbits.dataCols[orbit_type])

COM ['objId', 'q', 'e', 'inc', 'Omega', 'argPeri', 'tPeri', 'epoch', 'H', 'g', 'sed_filename']
KEP ['objId', 'a', 'e', 'inc', 'Omega', 'argPeri', 'meanAnomaly', 'epoch', 'H', 'g', 'sed_filename']
CART ['objId', 'x', 'y', 'z', 'xdot', 'ydot', 'zdot', 'epoch', 'H', 'g', 'sed_filename']


In [5]:
# and if 'standard but not quite the same' orbit column names are used, 
# the column names are swapped to these standard names. 

# Note that columns 'objId' = anything (including strings), and columns H / g / sed_filename are not required.
# If H is not specified, the value of 20 will be used
# If g is not specified, the value of 0.15 will be used
# If sed_filename is not specified (it should match a filename in $SIMS_MOVINGOBJECTS_DIR/data), one will be assigned.

help(orbits.assignSed)

Help on method assignSed in module lsst.sims.movingObjects.orbits:

assignSed(orbits, randomSeed=None) method of lsst.sims.movingObjects.orbits.Orbits instance
    Assign either a C or S type SED, depending on the semi-major axis of the object.
    P(C type) = 0 (a<2); 0.5*a - 1 (2<a<4); 1 (a > 4),
    based on figure 23 from Ivezic et al 2001 (AJ, 122, 2749).
    
    Parameters
    ----------
    orbits : pandas.DataFrame, pandas.Series or numpy.ndarray
       Array-like object containing orbital parameter information.
    
    Returns
    -------
    numpy.ndarray
        Array containing the SED type for each object in 'orbits'.



In [6]:
# The final orbit data is kept in a class attribute: orbits:
orbits.orbits[0:5]

,objId,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename
0,268897,2.834701,0.072095,15.926504,201.235164,212.481923,49028.823425,18.674,49353.160697,0.15,C.dat
1,268898,2.638647,0.168753,28.220880,137.114088,357.487466,49799.727938,20.761,49353.160697,0.15,S.dat
2,268899,1.772126,0.243933,21.180907,337.731210,254.147555,49227.454545,22.398,49353.160697,0.15,S.dat
3,268900,2.957352,0.116046,13.981384,105.319550,206.795452,49336.140572,16.995,49353.160697,0.15,C.dat
4,268901,2.909451,0.138365,22.418287,113.019746,254.782320,50138.141770,18.258,49353.160697,0.15,S.dat


### Connecting to (and reading from) the opsim data file.

In [7]:
# Look at the MAF documentation for more information on the lsst.sims.maf.db.OpsimDatabase class and methods.
opsimrun = 'baseline2018b'
opsdb = db.OpsimDatabase(os.path.join('db', opsimrun + '.db'))
colmap = getColMap(opsdb)

In [8]:
# Basically, we will just read all of these requested columns from the database: (for now we'll stick to night < 100)
reqcols = [colmap['mjd'], colmap['night'], colmap['ra'], colmap['dec'],
               'rotSkyPos', colmap['filter'], colmap['exptime'], colmap['seeingEff'],
               colmap['seeingGeom'], colmap['fiveSigmaDepth'], 'solarElong']
degreesIn = colmap['raDecDeg']
print(reqcols)

['observationStartMJD', 'night', 'fieldRA', 'fieldDec', 'rotSkyPos', 'filter', 'visitExposureTime', 'seeingFwhmEff', 'seeingFwhmGeom', 'fiveSigmaDepth', 'solarElong']


In [9]:
simdata = opsdb.fetchMetricData(reqcols, sqlconstraint='night < 100')

In [10]:
# For now, we apply a bit of a hack fix to standardize some column names:
simdata = mo.fixObsData(simdata, degreesIn=degreesIn)

In [11]:
# Simdata is a numpy recarray - let's just use pandas to make it easier to read here.
pd.DataFrame(simdata[0:3])

,observationStartMJD,night,fieldRA,fieldDec,rotSkyPos,filter,visitExposureTime,seeingFwhmEff,seeingFwhmGeom,fiveSigmaDepth,solarElong,ra,dec
0,59853.016794,1,305.088793,-24.889283,180.000000,z,30.0,0.692029,0.620848,23.348066,113.652908,305.088793,-24.889283
1,59853.018021,1,304.770694,35.966846,180.786087,z,30.0,1.182104,1.023690,22.308947,113.957998,304.770694,35.966846
2,59853.020185,1,305.482200,-62.802603,181.134222,z,30.0,0.766303,0.681901,23.178827,99.684211,305.482200,-62.802603


### Generating observations. 

There is more than one way to actually generate observations, but for now let's look at the linear interpolation approach (what I'd recommend at the moment). This is implemented in the class "LinearObs", which provides higher level methods that basically do the following: 

With LinearObs generating observations for each object goes as follows:  
* generate ephemerides between the start and end of the survey data, on stepsizes of tStep (typically, 2 hours)
* make linear interpolations between each of these steps, for every parameter returned by the ephemeris generation
* calculate the distance between the location of the object and the field pointing, for each observation
* if the distance is less than the specified 'rFov': count this observation as "observed" (if using the camera footprint, then the detailed camera footprint is overlaid for observations, and only objects which actually lie on active silicon are counted as "observed")
* for the pointings where the object was observed, then calculate trailing losses and write the output to disk (including a color term, appropriate for the filter of each observation).

(Note: there's plenty of room to improve on the procedure above, which could both speed things up and improve accuracy .. if this is something you're interested in contributing to, please consider writing a new Obs class, preferably inheriting from the BaseObs class).


In [12]:
# Using the "LinearObs" class.
usecamerafootprint = False
if usecamerafootprint:
    # Use the actual camera footprint.
    cameraFootprint = mo.LsstCameraFootprint()
else:
    # Or not (just use a circular fov)
    cameraFootprint = None
rFov = 1.75  # in degrees
obscode = 'I11'
ephMode = 'nbody' # nbody or 2body
obs = mo.LinearObs(cameraFootprint=cameraFootprint, rFov=rFov, obscode=obscode, timescale='TAI', ephMode=ephMode, 
                   timeCol=colmap['mjd'], seeingCol=colmap['seeingGeom'], visitExpTimeCol=colmap['exptime'])

In [13]:
# Set the orbits.
obs.setOrbits(orbits)
# Read the filter curves (to calculate the colors for each object)
filterlist = np.unique(simdata[colmap['filter']])
obs.readFilters(filterlist=filterlist)
# Calculate all colors ahead of time - this is just to create a dictionary early.
sednames = np.unique(obs.orbits.orbits['sed_filename'])
for sedname in sednames:
    obs.calcColors(sedname)
# Want to see the lsst colors?  (these are all x-V band colors)
obs.colors

{'C.dat': {'g': 0.28041793225569478,
  'i': -0.2926196728132453,
  'r': -0.177293845688272,
  'u': 1.5442111906705982,
  'y': -0.3028771664514025,
  'z': -0.29808899160965296},
 'S.dat': {'g': 0.36850606460945201,
  'i': -0.45720772665728049,
  'r': -0.26270911487248227,
  'u': 1.8626393418302385,
  'y': -0.4073469917610737,
  'z': -0.39986031775197262}}

In [14]:
# %%timeit -- a test run of this with 1999 TNOs and 100 nights (63472 visits) from baseline2018b.db == 
#   11min 36s ± 8.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# Generate the actual observations (using the linear interpolations) and write to output file.
tstep = 2.0 / 24.0  # tstep in units of days
obsFile = '_'.join([opsimrun, orbitfile.replace('.des', '').replace('.s3m', ''), 'obs.txt'])
print('Will write output to %s' % obsFile)
obs.run(simdata, obsFile, tstep=tstep)

Will write output to baseline2018b_mbas_2k_obs.txt
Generating ephemerides on a grid of 0.083333 day timesteps, then will extrapolate to opsim times.


In [15]:
# This is what the output file looks like:
!head -3 $obsFile

objId geo_dist ra dec magV dradt ddecdt velocity phase solarelon helio_dist h_lon h_lat true_anomaly time observationStartMJD night fieldRA fieldDec rotSkyPos filter visitExposureTime seeingFwhmEff seeingFwhmGeom fiveSigmaDepth solarElong ra dec magFilter dmagColor dmagTrail dmagDetect 
268898 2.6115829361 81.7129257927 -9.36992344077 26.2106616229 0.0877402892526 -0.122682962422 0.150829284658 18.5076776638 105.578968454 3.03750762386 59.6617104203 -27.5392723243 275.167701363 59854.3893056 59854.3893056 2 82.765139 -9.49079 258.413857923 y 30.0 0.764887329326 0.680737384706 21.947770741 104.790113577 82.765139 -9.49079 25.8033146312 -0.407346991761 0.0234595131402 0.0171995505918 
268898 2.56282834365 82.0460386645 -9.86121637559 26.156383885 0.0671060946542 -0.124070658177 0.141055864308 18.2377726223 108.419970146 3.03100850426 60.5001988444 -27.4550694163 275.914664777 59858.3709606 59858.3709606 6 82.765139 -9.49079 140.715222226 z 30.0 0.863522533984 0.761815522935 23.1031835023